In [1]:
from typing import Callable

# Um `thunk` é uma função que não recebe argumentos e retorna um valor.
# Vamos criar um tipo `Effect` que representa uma função que não recebe argumentos e retorna um valor.
# Note que ele é genérico do tipo A
type Effect[A] = Callable[[], A]

In [2]:
import random

# Isso é uma função normal, ela retorna uma int
def random_number() -> int:
    return random.randint(1, 100)

# Essa é uma função que retorna um `Effect` de int
# Ou seja, ela não executa a função imediatamente, mas retorna uma função que, 
# quando chamada, executará a função original e retornará o resultado.
def random_number_function() -> Effect[int]:
  return random_number

# Vamos criar um helper que simplesmente roda o `Effect` e imprime o resultado
def run[A](effect: Effect[A]) -> None:
    print(effect())

run(random_number_function())

58


In [3]:
# Porque diabos eu faria isso?
# Porque agora eu tenho uma função que pode ser executada mais tarde
# Em vez de modificar o resultado, eu consigo modificar a função em sim.

# Por exemplo, eu consigo repetir a execução de uma função várias vezes
# Mas note que `repeat` não executa a função imediatamente,
# ela retorna um novo efeito que retorna uma lista de A
def repeat[A](effect: Effect[A], times: int) -> Effect[list[A]]:
    def repeated_effect() -> list[A]:
        results: list[A] = []
        for _ in range(times):
            result = effect()
            results.append(result)
        return results

    return repeated_effect

# Agora a gente pode fazer composição de efeitos
# Podemos repetir a execução de `random_number_function` 5 vezes
run(repeat(random_number_function(), 5))

[28, 49, 93, 90, 50]


In [4]:
# Note que até eu fazer `run`, eu não estou executando nada. 
# E como repeat retorna um novo `Effect`, eu posso fazer isso várias vezes.

twice = repeat(random_number_function(), 2)
thrice = repeat(random_number_function(), 3)

run(twice)
run(thrice)

[36, 5]
[31, 3, 80]


In [5]:
# O Effect to typescript pode lidar com erros, mas vamos ignorar essa abordagem
# e fazer um efeito que dá um raise.

# Note que o `thunk` não possui argumentos,
# Mas uma função pode receber argumentos e retornar um `Effect` que usa esses argumentos.
def failing_effect(number: Effect[int]) -> Effect[int]:
    def effect() -> int:
        n = number()
        if n < 50:
            print("Simulating failure for number:", n)
            raise Exception("Random failure")
        return n

    return effect

# Note que eu precisa definir uma função internamente e dai retornar ela
# Isso acontece porque o Python não é uma boa linguagem para programação funcional
# Ela limita o lambdas a uma linha
run(failing_effect(random_number_function()))


66


In [6]:
# Novamente, o erro só acontece DEPOIS de chamar `run`
# Então vamos criar um efeito que tenta executar um efeito várias vezes até ter sucesso
def retry[A](effect: Effect[A], times: int) -> Effect[A]:
  def repeated_effect():
      for _ in range(times):
          try:
            return effect()
          except Exception:
            print("Effect failed, retrying...")
            continue
      raise Exception("Effect failed after retries")
  return repeated_effect

# Vamos tentar executar o efeito que falha 3 vezes
run(retry(failing_effect(random_number_function()), 3))

Simulating failure for number: 48
Effect failed, retrying...
51


In [7]:
# Puta merda, eu fico meio puto quando falha.
# Existe um padrão comum que é o `orElse`, ou seja, se falhar algo ali dentro, eu quero retornar um padrão
def or_else[A](effect: Effect[A], fallback: A) -> Effect[A]:
    def wrapped_effect() -> A:
        try:
            return effect()
        except Exception:
            print("Effect failed, returning fallback value:", fallback)
            return fallback
    return wrapped_effect


# Note que SEMPRE, SEMPRE eu retorno um `Effect`.
# Isso permite que todas as funções sejam compostas e encadeadas.

# Se falhar mesmo depois de tentar, vira 0
run(repeat(or_else(retry(failing_effect(random_number_function()), 2), 0), 20))

Simulating failure for number: 27
Effect failed, retrying...
Simulating failure for number: 30
Effect failed, retrying...
Simulating failure for number: 10
Effect failed, retrying...
Effect failed, returning fallback value: 0
Simulating failure for number: 29
Effect failed, retrying...
Simulating failure for number: 37
Effect failed, retrying...
Simulating failure for number: 20
Effect failed, retrying...
Effect failed, returning fallback value: 0
Simulating failure for number: 37
Effect failed, retrying...
Simulating failure for number: 32
Effect failed, retrying...
Effect failed, returning fallback value: 0
Simulating failure for number: 6
Effect failed, retrying...
Simulating failure for number: 26
Effect failed, retrying...
Simulating failure for number: 38
Effect failed, retrying...
Effect failed, returning fallback value: 0
Simulating failure for number: 28
Effect failed, retrying...
Simulating failure for number: 38
Effect failed, retrying...
Effect failed, returning fallback va

In [8]:
# O legal é que como a gente tem controle de QUANDO executar o efeito,
# podemos adicionar funções antes ou depois dele.

def telemetry[A](effect: Effect[A], text: str) -> Effect[A]:
    def wrapped_effect() -> A:
        print(f"Starting effect: {text}")
        result = effect()
        print(f"Effect completed with result: {result}")
        return result
    return wrapped_effect

run(telemetry(repeat(or_else(failing_effect(random_number_function()), 0), 10), "Repeat Effect"))

Starting effect: Repeat Effect
Simulating failure for number: 25
Effect failed, returning fallback value: 0
Simulating failure for number: 48
Effect failed, returning fallback value: 0
Effect completed with result: [53, 74, 99, 0, 71, 56, 72, 0, 96, 88]
[53, 74, 99, 0, 71, 56, 72, 0, 96, 88]


In [9]:
from typing import Tuple
from time import perf_counter

# O Effect não precisa sempre retornar A, ele pode retornar outros valores
def timed[A](effect: Effect[A]) -> Effect[Tuple[A, float]]:
    def timed_effect() -> Tuple[A, float]:
        start = perf_counter()
        result = effect()
        end = perf_counter()
        return (result, (end - start) * 10000)

    return timed_effect

# Vamos medir o tempo de execução do efeito
run(timed(repeat(or_else(retry(failing_effect(telemetry(random_number_function(), "Random number generator"),), 2), 0), 20)))

Starting effect: Random number generator
Effect completed with result: 79
Starting effect: Random number generator
Effect completed with result: 71
Starting effect: Random number generator
Effect completed with result: 58
Starting effect: Random number generator
Effect completed with result: 6
Simulating failure for number: 6
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 20
Simulating failure for number: 20
Effect failed, retrying...
Effect failed, returning fallback value: 0
Starting effect: Random number generator
Effect completed with result: 7
Simulating failure for number: 7
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 73
Starting effect: Random number generator
Effect completed with result: 45
Simulating failure for number: 45
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 77
Starting effect: Random number generator
Effect com

In [10]:
# Vai tomar no cu, tem tanto parentesis que eu fiquei perdido
# As funções estão todas encadeadas e eu não consigo ver o que está acontecendo
# E eu preciso ler de dentro pra fora

# Sabe que no UNIX a solução é encadear comandos com pipe?
# ex: `ls -l | grep py | wc -l` em vez de `wc -l (grep py (ls -l))`
# O toolz tem uma função chamada `pipe` que faz isso
from toolz import pipe # type: ignore

# Entretanto, o pipe faz o seguinte:
# pipe(data, f, g, h) === h(g(f(data)))

# E note que nossas funções recebem um `Effect` de primeiro argument
# portanto, precisamos criar uma função inline que recebe o `Effect` e retorna o resultado
# isso é o lambda, ele passa uma função que age em cima do resultado do `Effect`

result = pipe( # type: ignore
    random_number_function(),# O primeiro valor do pipe é um valor de verdade  # type: ignore
    lambda effect: telemetry(effect, "Random number generator"), # type: ignore
    failing_effect, # Nesse caso, o primeiro argumento é o Effect, então funciona 
    lambda effect: retry(effect, 2), # type: ignore
    lambda effect: or_else(effect, 0), # type: ignore
    lambda effect: repeat(effect, 20), # type: ignore
    timed, # como não precisa de outros argumentos, podemos passar diretamente
    run # o mesmo para o run
)

# Note que agora eu consigo ler de fora pra dentro
# E eu consigo ver o que está acontecendo em cada etapa
# Como se fosse um pipeline de dados

Starting effect: Random number generator
Effect completed with result: 39
Simulating failure for number: 39
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 26
Simulating failure for number: 26
Effect failed, retrying...
Effect failed, returning fallback value: 0
Starting effect: Random number generator
Effect completed with result: 37
Simulating failure for number: 37
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 41
Simulating failure for number: 41
Effect failed, retrying...
Effect failed, returning fallback value: 0
Starting effect: Random number generator
Effect completed with result: 73
Starting effect: Random number generator
Effect completed with result: 11
Simulating failure for number: 11
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 38
Simulating failure for number: 38
Effect failed, retrying...
Effect failed, returning fallb

In [11]:
# Entretanto, a gente ainda consegue fazer melhor
# Note que o `pipe` espera uma função que recebe somente um argumento, 
# mas nossas funções recebem um `Effect` como primeiro argumento e as vezes passam outro `Effect` como segundo argumento.

# Para isso existe uma técnica chamada `partial application`
# que é basicamente criar uma função que recebe um dos argumentos e retorna uma nova função que recebe o restante dos argumentos.

# Como:
# sum(a, b) === sum(a)(b)
from toolz import partial # type: ignore

# partial retorna uma função que recebe menos argumentos do que a função original
# e quando essa função é chamada, ela chama a função original com os argumentos restantes.

# Ou seja, `with_repeat` vai retornar uma função que recebe um `Effect`
def with_repeat[A](times: int) -> Callable[[Effect[A]], Effect[list[A]]]:
  return partial(repeat, times=times) # type: ignore

# with_repeat(5)(random_number_function()) == repeat(random_number_function(), 5)

# Vamos fazer essa versão das nossas funções que recebem um `Effect` como primeiro argumento
def with_retry[A](times: int) -> Callable[[Effect[A]], Effect[A]]:
    return partial(retry, times=times) # type: ignore

def with_or_else[A](fallback: A) -> Callable[[Effect[A]], Effect[A]]:
    return partial(or_else, fallback=fallback) # type: ignore

def with_telemetry[A](text: str) -> Callable[[Effect[A]], Effect[A]]:
    return partial(telemetry, text=text) # type: ignore


pipe( # type: ignore
    random_number_function(), # O primeiro valor do pipe é um valor de verdade  # type: ignore
    with_telemetry("Random number generator"), # type: ignore
    failing_effect, # Nesse caso, o primeiro argumento é o Effect, então funciona 
    with_retry(2), # type: ignore
    with_or_else(0), # type: ignore
    with_repeat(20), # type: ignore
    timed, # como não precisa de outros argumentos, podemos passar diretamente
    run # o mesmo para o run
)

# Você pode ler pensando:
# Rodar esse efeito com telemetria,
# se falhar, tentar mais duas vezes,
# se falhar, retornar 0,
# repetir 20 vezes,
# medir o tempo de execução


Starting effect: Random number generator
Effect completed with result: 8
Simulating failure for number: 8
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 91
Starting effect: Random number generator
Effect completed with result: 81
Starting effect: Random number generator
Effect completed with result: 41
Simulating failure for number: 41
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 84
Starting effect: Random number generator
Effect completed with result: 82
Starting effect: Random number generator
Effect completed with result: 2
Simulating failure for number: 2
Effect failed, retrying...
Starting effect: Random number generator
Effect completed with result: 52
Starting effect: Random number generator
Effect completed with result: 60
Starting effect: Random number generator
Effect completed with result: 20
Simulating failure for number: 20
Effect failed, retrying...
Starting effect: Ran

In [12]:
# Duas funções muito úteis:
# map
# Ele é LITERALMENTE a mesma coisa que um map de list. 
# Pega o resultado do efeito, e aplica uma função
def map_effect[A, B](effect: Effect[A], func: Callable[[A], B]) -> Effect[B]:
    def mapped_effect() -> B:
        result = effect()
        return func(result)
    return mapped_effect 

# Isso permite eu utilizar lambdas diretamente
run(map_effect(random_number_function(), lambda x: f"Número aleatório: {x}"))

# Mas e se a função que eu quero aplicar também for um efeito?
# Nesse caso, a gente "amassa" os dois, geralmente chamam isso de flat_map
def flat_map_effect[A, B](effect: Effect[A], func: Callable[[A], Effect[B]]) -> Effect[B]:
    return func(effect())  # Chama o efeito original, depois chama o efeito que a função recebeu

def duplicate_effect(effect: Effect[int]) -> Effect[int]:
    def duplicated_effect() -> int:
        return effect() * 2
    return duplicated_effect

run(flat_map_effect(random_number_function, duplicate_effect))


Número aleatório: 83
22


In [13]:
# type: ignore
# Uma coisa chata é a necessida de definir uma função para cada coisa
def gen(generator_func):
    def run_effect():
        generator = generator_func()
        
        try:
            # Start the generator and get the first yielded Effect
            current_effect = next(generator)
            
            while True:
                # Run the yielded Effect to get its result
                result = current_effect()
                
                try:
                    # Send the result back to the generator and get the next Effect
                    current_effect = generator.send(result)
                except StopIteration as e:
                    # Generator completed, return its final value
                    return e.value
                    
        except StopIteration as e:
            # Generator completed without yielding anything
            return e.value
    
    return run_effect

@gen
def my_effect():
    a = yield random_number_function()
    b = yield random_number_function()

    if a > b:
        return f"Primeiro número era maior: {a}"
    else:
        return f"Segundo numero era maior: {b}"

# Ainda é um efeito! Então podemos fazer
run(repeat(my_effect, 2))

['Primeiro número era maior: 19', 'Primeiro número era maior: 12']


In [ ]:
# Vamos tentar fazer um exemplo real, primeiro algumas funções e utilies
from typing import TypedDict, Optional
from time import sleep
from termcolor import colored

emails = [
    "john.doe@example.com",
    "jane.smith@example.com",
    "alice.johnson@example.com",
    "bob.brown@example.com",
    "charlie.davis@example.com",
    "emily.clark@example.com",
    "frank.harris@example.com",
    "grace.lee@example.com",
    "henry.miller@example.com",
    "ivy.wilson@example.com",
]

class User(TypedDict):
    id: int
    whatsapp: Optional[str]
    email: str

def get_user(id: int) -> Effect[User | Exception]:
    # 50% de chance de dar erro na API
    def wrapped_effect():
        if random.random() < 0.5:
            return Exception("Erro ao buscar usuário")
        return User({
            "id": id,
            "whatsapp": random.choice([None, "1234-5678"]),
            "email": emails[id-1]
        })
    
    return wrapped_effect

def retry_if_exception[A](effect: Effect[A], retries: int) -> Effect[A | Exception]:
    def wrapped_effect():
        for _ in range(retries):
            result = effect()
            if not isinstance(result, Exception):
                return result
        return Exception("Max retries exceeded")
    return wrapped_effect

def send_zap_zap(to: str, message: str) -> Effect[str | Exception]:
    def wrapped_effect():
        sleep(random.random())
        if random.random() < 0.5:
            return Exception("Erro ao enviar mensagem pelo WhatsApp")
        return(f"Zap enviada para {to}")
    return wrapped_effect

def send_email(to: str, subject: str) -> Effect[str | Exception]:
    def wrapped_effect():
        sleep(random.random())
        if random.random() < 0.5:
            return Exception("Erro ao enviar email")
        return(f"Email enviado para {to}")
    return wrapped_effect

# Note que aqui a gente não muda o tipo do efeito
# É só uma função que le o resultado e retorna ele mesmo
def debug_to_gcp[A](effect: Effect[A], message: str) -> Effect[A]: 
    def wrapped_effect():
        result = effect()
        print(colored(f"DEBUG: {message}", 'green'))
        return result
    return wrapped_effect

def log_error[A](effect: Effect[A]) -> Effect[A]:
    def wrapped_effect():
        result = effect()
        if isinstance(result, Exception):
            print(colored(f"ERROR: {result}", "red"))
        return result
    return wrapped_effect

  
def telemetry_effect[A](effect: Effect[A], message: str) -> Effect[A]:
    def wrapped_effect() -> A:
        timed_effect = timed(effect)

        def time_message(pair: tuple[A, float]) -> Effect[A]:
            msg = f"{message}: {pair[1]/10000:.2f} seconds"
            return debug_to_gcp(lambda: pair[0], msg)

        logged = flat_map_effect(timed_effect, time_message)
        err_logged = log_error(logged)
        return err_logged()
    return wrapped_effect


def zap_user(number: str, message: str):
    return telemetry_effect(send_zap_zap(number, message), f"zap_user: {number}")

def email_user(email: str, subject: str):
    return telemetry_effect(send_email(email, subject), f"email_user: {email}")


def run_all_effects[A](effects: list[Effect[A]]) -> None:
  results: list[A] = []
  for effect in effects:
      result = effect()
      results.append(result)

  for result in results:
      print(result)


In [73]:
# type: ignore

# Agora força o olho, e imagina que `yield` na verdade é `await`
# é a mesma coisa

def main(id: int):
      
  @gen
  def generator():
      user = yield retry_if_exception(get_user(id), 10)


      # Aqui podemos utilizar um fluxo normal
      if isinstance(user, Exception):
          return f'Error ao buscar usuário {id}'

      if user["whatsapp"]:
          response = yield retry_if_exception(zap_user(user["whatsapp"], "Olá, você tem uma nova mensagem!"), 2)

      elif user["email"]:
          response = yield retry_if_exception(email_user(user["email"], "Nova mensagem recebida"), 2)

      if isinstance(response, Exception):
          return f"Erro ao enviar mensagem para usuário id {user['id']}"
      
      return response

  return generator


run_all_effects(main(i) for i in range(10))


DEBUG: email_user: ivy.wilson@example.com: 0.90 seconds
ERROR: Erro ao enviar email
DEBUG: email_user: ivy.wilson@example.com: 0.52 seconds
DEBUG: email_user: john.doe@example.com: 0.58 seconds
ERROR: Erro ao enviar email
DEBUG: email_user: john.doe@example.com: 0.91 seconds
DEBUG: zap_user: 1234-5678: 0.84 seconds
DEBUG: email_user: alice.johnson@example.com: 0.54 seconds
DEBUG: zap_user: 1234-5678: 0.58 seconds
ERROR: Erro ao enviar mensagem pelo WhatsApp
DEBUG: zap_user: 1234-5678: 0.71 seconds
ERROR: Erro ao enviar mensagem pelo WhatsApp
DEBUG: email_user: charlie.davis@example.com: 0.02 seconds
ERROR: Erro ao enviar email
DEBUG: email_user: charlie.davis@example.com: 0.72 seconds
DEBUG: email_user: emily.clark@example.com: 0.72 seconds
ERROR: Erro ao enviar email
DEBUG: email_user: emily.clark@example.com: 0.55 seconds
DEBUG: email_user: frank.harris@example.com: 0.02 seconds
ERROR: Erro ao enviar email
DEBUG: email_user: frank.harris@example.com: 0.06 seconds
ERROR: Erro ao envia